<a href="https://colab.research.google.com/github/NischalSuresh/lesion_detection/blob/main/lesion_superimposition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import shutil
shutil.copy('/content/drive/MyDrive/Projects/DermViz/Synthetic_data/masks.zip', '/content/masks.zip')
shutil.copy('/content/drive/MyDrive/Projects/DermViz/Synthetic_data/crops.zip', '/content/crops.zip')
shutil.copy('/content/drive/MyDrive/Projects/DermViz/Synthetic_data/backbody_aug.zip', '/content/backbody_aug.zip')


'/content/backbody_aug.zip'

In [3]:
!unzip /content/masks.zip -d /content
!unzip /content/crops.zip -d /content
!unzip /content/backbody_aug.zip -d /content

Archive:  /content/masks.zip
   creating: /content/masks/
  inflating: /content/masks/F14_2016_09_09.jpg_crop_19.jpg  
  inflating: /content/masks/14_7085.JPG_crop_3.jpg  
  inflating: /content/masks/F14_2016_09_09.jpg_crop_25.jpg  
  inflating: /content/masks/G16_2003_03_31_Regist.jpg_crop_18.jpg  
  inflating: /content/masks/G16_2003_03_31_Regist.jpg_crop_30.jpg  
  inflating: /content/masks/O14_2010_03_08_Regist.jpg_crop_1.jpg  
  inflating: /content/masks/R16_2008_12_22.jpg_crop_12.jpg  
  inflating: /content/masks/G16_2003_03_31_Regist.jpg_crop_24.jpg  
  inflating: /content/masks/F14_2016_09_09.jpg_crop_2.jpg  
  inflating: /content/masks/U16_2011_02_14.jpg_crop_2.jpg  
  inflating: /content/masks/13_7097.JPG_crop_5.jpg  
  inflating: /content/masks/O14_2015_10_12.jpg_crop_0.jpg  
  inflating: /content/masks/14_7097.JPG_crop_0.jpg  
  inflating: /content/masks/F16_2007_02_15_Regist.jpg_crop_9.jpg  
  inflating: /content/masks/F14_2010_10_11_Regist.jpg_crop_4.jpg  
  inflating: /c

In [4]:
import os
print(len(os.listdir("/content/crops")))
print(len(os.listdir("/content/masks")))
print(len(os.listdir("/content/backbody_aug")))

735
735
284


In [5]:
# Disable the idle timeout
from IPython.display import Javascript
def disable_autoplay():
  disable_str = """
    function ClickConnect(){
      console.log("Working");
      document.querySelector("colab-toolbar-button#connect").click()
    }
    setInterval(ClickConnect,60000)
  """
  display(Javascript(disable_str))
disable_autoplay()

<IPython.core.display.Javascript object>

In [6]:
from IPython.display import display, HTML
display(HTML(
"""
<a target="_blank" href="https://colab.research.google.com/github/facebookresearch/segment-anything/blob/main/notebooks/predictor_example.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
"""
))

import torch
import torchvision
# print("PyTorch version:", torch.__version__)
# print("Torchvision version:", torchvision.__version__)
# print("CUDA is available:", torch.cuda.is_available())
import sys
!{sys.executable} -m pip install opencv-python matplotlib
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

# !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-6jwnv7ar
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-6jwnv7ar
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36586 sha256=c86b8e79ae6dd065c608006c79b9072452cd552320736edac0d56049f84740a9
  Stored in directory: /tmp/pip-ephem-wheel-cache-43hfwe1z/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything


In [7]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

Wed Sep 20 14:21:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import random
import os
import sys
import xml.etree.ElementTree as ET
import xml.dom.minidom
import csv

from google.colab.patches import cv2_imshow
from tqdm import tqdm
from segment_anything import sam_model_registry, SamPredictor

In [9]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))

from google.colab import drive
drive.mount('/content/drive')

shared_drive_path = "/content/drive/MyDrive"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Example image

In [10]:
# image_folder_path = '/content/drive/MyDrive/Projects/DermViz/Synthetic_data/backbody_aug'
image_folder_path = '/content/backbody_aug'

image_snippets = os.listdir(image_folder_path)

count=0
master_csv_rows = []

sam_checkpoint = "/content/drive/MyDrive/Projects/DermViz/Synthetic_data/SAM_weights/sam_vit_b_01ec64.pth"
model_type = "vit_b"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)

In [11]:
import numpy as np

def generate_moles(num_samples):
    mean_num_moles = 30
    std_dev = 15
    min_num_moles = 20
    max_num_moles = 100

    # Generate 6000 moles using the clipped Gaussian distribution
    moles = np.random.normal(mean_num_moles, std_dev, num_samples)
    moles = np.clip(moles, min_num_moles, max_num_moles)
    moles = moles.astype(int)

    return moles

# Generate 5000 moles using the function
num_samples = 6000
num_moles_list = generate_moles(num_samples)

# Print the first 10 values
print(num_moles_list[:100])

[21 28 37 51 41 30 20 27 43 20 43 26 22 30 20 65 20 20 20 20 30 20 20 48
 21 39 43 24 31 34 29 30 32 51 66 20 20 20 46 49 20 22 46 20 23 34 46 20
 42 24 36 30 20 20 50 48 21 44 20 34 33 40 32 59 51 22 29 38 28 25 26 23
 58 20 32 33 45 20 21 29 43 20 28 35 22 45 20 33 38 20 20 20 20 45 21 28
 20 51 20 49]


In [12]:
# mole_snippets_folder = '/content/drive/MyDrive/Projects/DermViz/Synthetic_data/crops'
mole_snippets_folder = '/content/drive/MyDrive/Projects/DermViz/Synthetic_data/crops'

# mole_snippets_segmentation_folder = '/content/drive/MyDrive/Projects/DermViz/Synthetic_data/masks'
mole_snippets_segmentation_folder = '/content/masks'

# mole_snippets_segmentation_dilated_folder = '/content/drive/MyDrive/Lesion Detection/final_dilated_masks/'
mole_snippets = os.listdir(mole_snippets_folder)
mole_snippets_segmentation = os.listdir(mole_snippets_segmentation_folder)
# mole_snippets_segmentation_dilated = os.listdir(mole_snippets_segmentation_dilated_folder)
master_csv_file_path = '/content/drive/MyDrive/Projects/DermViz/Synthetic_data/master.csv'

num_images = 10

In [13]:
print(len(os.listdir(mole_snippets_folder)))

735


In [14]:
import pdb

In [ ]:
for image_snippet_name in tqdm(image_snippets):
    # Read the backbody image
    image_snippet_path = os.path.join(image_folder_path, image_snippet_name)
    image = cv2.imread(image_snippet_path)
    if image is not None:
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Read the original image
    original_image = cv2.imread(image_snippet_path)

    # Convert the images to RGB color space
    if original_image is not None:
      original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

    sys.path.append("..")
    predictor.set_image(image)
    height, width, _ = image.shape

    k = 10
    x_mid, y_mid = width // 2, height // 2  # Midpoint

    input_point = np.array([
        [x_mid - width // k, y_mid - height // k],
        [x_mid + width // k, y_mid - height // k],
        [x_mid + width // k, y_mid + height // k],
        [x_mid - width // k, y_mid + height // k],
        [x_mid, y_mid]
    ])
    input_label = np.array([1, 1, 1, 1, 1])

    masks, scores, logits = predictor.predict(
        point_coords=input_point,
        point_labels=input_label,
        multimask_output=True,
    )

    for i, (mask, score) in enumerate(zip(masks, scores)):
      masks, scores, logits = predictor.predict(
        point_coords=input_point,
        point_labels=input_label,
        multimask_output=True,
    )

    if masks.shape[0] > 0:
        best_mask_index = np.argmax(scores)
        best_score = scores[best_mask_index]

    # Convert the mask to uint8 data type
    best_mask = (masks[best_mask_index] * 255).astype(np.uint8)
    backbody_image = best_mask

    original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

    # Resize and convert backbody image to color
    backbody_resized = cv2.resize(backbody_image, (500, 500))
    backbody_color = cv2.cvtColor(backbody_resized, cv2.COLOR_GRAY2BGR)

    # Erode the result image (segmentation mask)
    kernel = np.ones((20, 20), np.uint8)  # Adjust the kernel size as needed
    result_image_eroded = cv2.erode(backbody_resized, kernel, iterations=3)

    def superimpose_moles(backbody_image, mole_snippets, num_moles):
        mole_info = []

        result_image = cv2.resize(backbody_image, (1000, 1000))  # Resize backbody image mask
        original_image_resized = cv2.resize(original_image, (1000, 1000))  # Resize backbody image
        segmentation_mask = np.zeros_like(original_image_resized)  # Creating a segmentation mask

        # Erode the result image (segmentation mask)
        kernel = np.ones((20, 20), np.uint8)  # Adjust the kernel size as needed
        eroded_mask = cv2.erode(result_image, kernel, iterations=3)

        # Select random moles
        selected_moles = random.sample(mole_snippets, num_moles)

        for mole_file in selected_moles:
            mole_path = os.path.join(mole_snippets_folder, mole_file)

            # Load the mole snippet
            mole_snippet = cv2.imread(mole_path, cv2.IMREAD_UNCHANGED)

            # Check if the mole snippet has non-empty dimensions
            if mole_snippet is not None and mole_snippet.shape[0] > 0 and mole_snippet.shape[1] > 0:
                # Randomly determine the size of the mole snippet
                mole_size = random.randint(8, 30)
                mole_snippet = cv2.resize(mole_snippet, (mole_size, mole_size))

            # Find non-zero pixels in the eroded mask
            nonzero_indices = np.nonzero(eroded_mask)

            # Select random coordinates from the non-zero pixels
            rand_index = random.randint(0, len(nonzero_indices[0]) - 1)
            mole_x = nonzero_indices[1][rand_index]
            mole_y = nonzero_indices[0][rand_index]

            # Load the ground truth segmentation
            mole_segmentation_file = mole_file
            # mole_segmentation_dilated_file = mole_file.split(".")[0] + '_Dilated.png'
            mole_segmentation_path = os.path.join(mole_snippets_segmentation_folder, mole_segmentation_file)
            # mole_segmentation_dilated_path = os.path.join(mole_snippets_segmentation_dilated_folder, mole_segmentation_dilated_file)
            mole_segmentation = cv2.imread(mole_segmentation_path, cv2.IMREAD_UNCHANGED)
            # mole_segmentation_dilated = cv2.imread(mole_segmentation_dilated_path, cv2.IMREAD_UNCHANGED)

            # Resize the ground truth segmentation to match the changes made to the original mole snippet
            if mole_snippet is None:
              print(mole_file)

            if mole_segmentation is None:
              print(mole_segmentation_file)

            # if mole_segmentation_dilated is None:
            #   print(mole_segmentation_dilated_file)
            # print(mole_segmentation_path)
            mole_segmentation_resized = cv2.resize(mole_segmentation, (mole_snippet.shape[1], mole_snippet.shape[0]))
            _, mole_segmentation_resized = cv2.threshold(mole_segmentation_resized, 127, 255, cv2.THRESH_BINARY)
            # mole_segmentation_dilated_resized = cv2.resize(mole_segmentation_dilated, (mole_snippet.shape[1], mole_snippet.shape[0]))

            # Find the bounding box coordinates based on the resized ground truth segmentation
            bbox_x = mole_x + np.min(np.nonzero(mole_segmentation)[1])
            bbox_y = mole_y + np.min(np.nonzero(mole_segmentation)[0])
            bbox_width = np.max(np.nonzero(mole_segmentation)[1]) - np.min(np.nonzero(mole_segmentation)[1])
            bbox_height = np.max(np.nonzero(mole_segmentation)[0]) - np.min(np.nonzero(mole_segmentation)[0])

            # Draw the bounding box on the original image
            # cv2.rectangle(original_image_resized, (bbox_x, bbox_y), (bbox_x + bbox_width, bbox_y + bbox_height), (0, 255, 0), 2)

            # Draw the bounding box on the segmenation mask
            # cv2.rectangle(segmentation_mask, (bbox_x, bbox_y), (bbox_x + bbox_width, bbox_y + bbox_height), 255, -1)

            # Calculate the centroid coordinates
            centroid_x = bbox_x + bbox_width // 2
            centroid_y = bbox_y + bbox_height // 2

            # Draw a circle at the centroid position on the original image
            # cv2.circle(original_image_resized, (centroid_x, centroid_y), 3, (255, 0, 0), -1)

            # Store mole snippet information
            mole_info.append((mole_file, (mole_x, mole_y, mole_x + mole_snippet.shape[1], mole_y + mole_snippet.shape[0]),
                              (centroid_x, centroid_y)))

            # Superimpose the mole snippet on the backbody image
            # pdb.set_trace()
            if mole_snippet.shape[2] == 4:
                # If mole snippet has an alpha channel
                alpha_mask = mole_snippet[:, :, 3] / 255.0
                for c in range(3):
                    # Get the non-zero pixels of the mole snippet
                    mole_pixels = mole_snippet[:, :, c] * alpha_mask
                    # Calculate the region of interest for superimposition
                    roi = original_image_resized[mole_y:mole_y + mole_snippet.shape[0], mole_x:mole_x + mole_snippet.shape[1], c]
                    # Update only the non-zero pixels of the ROI with mole pixels
                    np.copyto(roi, mole_pixels, where=(mole_segmentation_resized > 0))
                    # Update the result image with the updated ROI
                    original_image_resized[mole_y:mole_y + mole_snippet.shape[0], mole_x:mole_x + mole_snippet.shape[1], c] = roi
                    # Update the segmentation mask with the white pixels of the mole snippet
                    mole_pixels_resized = cv2.resize(mole_pixels, (mole_snippet.shape[1], mole_snippet.shape[0]))
                    segmentation_mask[mole_y:mole_y + mole_snippet.shape[0], mole_x:mole_x + mole_snippet.shape[1], c] = np.where(mole_segmentation_resized > 0, 255, segmentation_mask[mole_y:mole_y + mole_snippet.shape[0], mole_x:mole_x + mole_snippet.shape[1], c])
            else:
                # If mole snippet does not have an alpha channel, superimpose RGB channels only
                for c in range(3):
                    # Get the non-zero pixels of the mole snippet
                    mole_pixels = mole_snippet[:, :, c]
                    # Calculate the region of interest for superimposition
                    roi = original_image_resized[mole_y:mole_y + mole_snippet.shape[0], mole_x:mole_x + mole_snippet.shape[1], c]
                    mole_pixels_resized = cv2.resize(mole_pixels, roi.shape[::-1])
                    mole_segmentation_resized = cv2.resize(mole_segmentation_resized, roi.shape[::-1])
                    # print("roi shape =", roi.shape)
                    # print("mole_pixels_resized shape =", mole_pixels_resized.shape)
                    # print("mole_segmentation_resized shape =", mole_segmentation_resized.shape)
                    try:
                      np.copyto(roi, mole_pixels_resized, where=(mole_segmentation_resized > 0))
                    except:
                      print("mole_file =", mole_file)
                      pdb.set_trace()
                    original_image_resized[mole_y:mole_y + mole_snippet.shape[0], mole_x:mole_x + mole_snippet.shape[1], c] = roi
                    segmentation_mask[mole_y:mole_y + mole_snippet.shape[0], mole_x:mole_x + mole_snippet.shape[1], c] = np.where(mole_segmentation_resized > 0, 255, segmentation_mask[mole_y:mole_y + mole_snippet.shape[0], mole_x:mole_x + mole_snippet.shape[1], c])
        return original_image_resized, mole_info, segmentation_mask

    mole_info_list = []

    output_dir = '/content/drive/MyDrive/Projects/DermViz/Synthetic_data/XML/'

    for i in range(num_images):
        # Superimpose moles on the backbody image
        # print(".")
        num_moles = num_moles_list[count]

        new_image, mole_info, segmentation_mask = superimpose_moles(backbody_image, mole_snippets, num_moles)
        mole_info_list.append(mole_info)

        # Create the root element
        root = ET.Element("processedImageResults")

        # Add image file name and pixels per CM
        image_file_name = ET.SubElement(root, "imageFile")
        image_file_name.text = image_snippet_name

        # Create cues element
        cues = ET.SubElement(root, "cues")

        # Display the new image with bounding boxes and centroids
        for mole in mole_info:
            mole_file, bounding_box, centroid = mole

            # Create a cue element for each mole
            cue = ET.SubElement(cues, "cue")

            # Add mole image name
            mole_image_name = ET.SubElement(cue, "moleImageName")
            mole_image_name.text = mole_file

            # Add bounding box information
            bounding_box_element = ET.SubElement(cue, "boundingBox")
            x0 = ET.SubElement(bounding_box_element, "x0")
            x0.text = str(bounding_box[0])
            y0 = ET.SubElement(bounding_box_element, "y0")
            y0.text = str(bounding_box[1])
            x1 = ET.SubElement(bounding_box_element, "x1")
            x1.text = str(bounding_box[2])
            y1 = ET.SubElement(bounding_box_element, "y1")
            y1.text = str(bounding_box[3])

            # Add centroid information
            centroid_element = ET.SubElement(cue, "centroid")
            x = ET.SubElement(centroid_element, "x")
            x.text = str(centroid[0])
            y = ET.SubElement(centroid_element, "y")
            y.text = str(centroid[1])

        # Create the XML tree
        tree = ET.ElementTree(root)

        # Save the XML to a file
        xml_filename = image_snippet_name.split('.')[0] + '_' + str(i) + '.xml'
        with open(output_dir + xml_filename, "w") as file:
            document = xml.dom.minidom.parseString(ET.tostring(root))
            file.write(document.toprettyxml(indent="    "))

        # Convert the segmentation mask to a binary image with a single channel
        segmentation_mask_binary = cv2.cvtColor(segmentation_mask, cv2.COLOR_BGR2GRAY)
        _, segmentation_mask_binary = cv2.threshold(segmentation_mask_binary, 1, 255, cv2.THRESH_BINARY)

        # Find contours in the segmentation mask
        contours, _ = cv2.findContours(segmentation_mask_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Draw the contours on the segmentation mask
        mask_contours = np.zeros_like(segmentation_mask)
        cv2.drawContours(mask_contours, contours, -1, (0, 255, 0), 2)

        # Create a mask image with the same size as the new image
        mask = np.zeros_like(new_image)

        # Draw the lesion edges as white on the mask
        cv2.drawContours(mask, contours, -1, (255, 255, 255), 2)

        # Resize the mask to match the dimensions of the new image
        mask = cv2.resize(mask, (new_image.shape[1], new_image.shape[0]))

        # # Apply Gaussian blur only to the lesion edges by blending with a blurred version of the image
        # blurred_image = cv2.GaussianBlur(new_image, (15, 15), 0)
        # blended_image = cv2.bitwise_and(blurred_image, mask) + cv2.bitwise_and(new_image, cv2.bitwise_not(mask))

        # Save the new image
        cv2.imwrite(f'/content/drive/MyDrive/Projects/DermViz/Synthetic_data/superimposed_back/'+image_snippet_name.split('.')[0]+'_'+str(i)+'.jpg', new_image)
        cv2.imwrite(f'/content/drive/MyDrive/Projects/DermViz/Synthetic_data/superimposed_segmentation/'+image_snippet_name.split('.')[0]+'_'+str(i)+'_segmented.jpg', segmentation_mask_binary)
        new_image_filename = image_snippet_name.split('.')[0]+'_'+str(i)+'.jpg'
        segmentation_mask_filename = image_snippet_name.split('.')[0]+'_'+str(i)+'_segmented.jpg'

        # Append the CSV row
        master_csv_rows.append([count, new_image_filename, segmentation_mask_filename, xml_filename])
        count = count+1

100%|██████████| 284/284 [31:28<00:00,  6.65s/it]


In [ ]:
print(count)

1005


In [ ]:
# Write the CSV rows to the master CSV file
with open(master_csv_file_path, 'w', newline='') as csv_file:
  writer = csv.writer(csv_file)
  writer.writerow(['Index','Image', 'Segmentation Mask', 'XML'])
  writer.writerows(master_csv_rows)